<a href="https://colab.research.google.com/github/kevin-rn/Grounding-LM/blob/main/abstractive_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab setup

In [15]:
import os
from google.colab import drive
drive.mount('/content/drive')

outdir = './results'
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [2]:
%cp -R ./drive/MyDrive/Grounding_LM/data ./

# Packages & Imports

In [1]:
%pip install -q pytorch-lightning
%pip install -q transformers
%pip install -q datasets
%pip install -q openai
%pip install -q tiktoken
%pip install -U -q pip setuptools wheel
%pip install -U -q spacy
!python -m spacy download en_core_web_sm
# %pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
                                              0.0/12.8 MB ? eta -:--:--
                                              0.0/12.8 MB ? eta -:--:--
                                              0.0/12.8 MB ? eta -:--:--
                                             0.1/12.8 MB 469.7 kB/s eta 0:00:28
                                             0.1/12.8 MB 722.1 kB/s eta 0:00:18
     ---                                      1.1/12.8 MB 5.2 MB/s eta 0:00:03
     ------                                   2.2/12.8 MB 8.8 MB/s eta 0:00:02
    

In [1]:
from datasets import load_dataset, load_from_disk
import pandas as pd
import tiktoken
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.optim import AdamW
from tqdm.auto import tqdm
import openai
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import random
import spacy
from spacy.lang.en import English

nlp = spacy.load("en_core_web_sm")

c:\Users\kevin\miniconda3\envs\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

## Dataset

In [11]:
# Download Dataset
def download_data(dataname):
    match dataname:
        case "xsum":
            data = load_dataset("xsum")
        case "cnn_dailymail":
            data = load_dataset("cnn_dailymail", "3.0.0")
        case _ :
            data = load_dataset("webis/tldr-17")

    # Split Dataset
    df_train = pd.DataFrame(data=data['train'])
    df_val = pd.DataFrame(data=data['validation'])
    df_test = pd.DataFrame(data=data['test'])

    # Rename columns for later usage
    df_train.columns = ['text', 'summary', 'id']
    df_val.columns = ['text', 'summary', 'id']
    df_test.columns = ['text', 'summary', 'id']

    # Save to data folder (inside Grounding_LM folder)
    df_train.to_csv(f'data/{dataname}/train.csv', index=False)
    df_val.to_csv(f'data/{dataname}/validation.csv', index=False)
    df_test.to_csv(f'data/{dataname}/test.csv', index=False)

# Read Dataset from folder
def read_data(dataname):
    df_train = pd.read_csv(f'data/{dataname}/train.csv')
    df_val = pd.read_csv(f'data/{dataname}/validation.csv')
    df_test = pd.read_csv(f'data/{dataname}/test.csv')
    df_test[['text', 'summary']] = df_test[['text', 'summary']].astype(str)
    return df_train, df_val, df_test

In [12]:
dataset = "xsum"
# download_data(dataset)
df_train, df_val, df_test = read_data(dataname=dataset)
df_train.head()

,text,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984


## Custom Datamodule

In [6]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, text_max_len = 512, summary_max_len = 128):
        self.data = data
        self.tokenizer = tokenizer
        self.text_max_len = text_max_len
        self.summary_max_len = summary_max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        text = self.data.iloc[idx]['text']
        summary = self.data.iloc[idx]['summary']
        id = self.data.iloc[idx]['id']

        text_encoding = self.tokenizer(
            text,
            max_length=self.text_max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = self.tokenizer(
            summary,
            max_length=self.summary_max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = summary_encoding['input_ids']
        labels[labels == 0] = -100

        return {
            'text': text,
            'summary': summary,
            'id': id,
            'input_ids': text_encoding['input_ids'],
            'attention_mask': text_encoding['attention_mask'],
            'labels': labels.flatten(),
            'labels_attention_mask': summary_encoding['attention_mask']
        }

In [7]:
class CustomDataModule(pl.LightningDataModule):
    def __init__(self, df_train, df_val, df_test, tokenizer, batch = 8, text_max_len = 512, summary_max_len = 128):
        super().__init__()
        self.df_train = df_train
        self.df_val = df_val
        self.df_test = df_test
        self.tokenizer = tokenizer
        self.batch = batch
        self.text_max_len = text_max_len
        self.summary_max_len = summary_max_len

    def setup(self, stage=None):
      self.train_dataset = CustomDataset(self.df_train, self.tokenizer, self.text_max_len, self.summary_max_len)
      self.val_dataset = CustomDataset(self.df_val, self.tokenizer, self.text_max_len, self.summary_max_len)
      self.test_dataset = CustomDataset(self.df_test, self.tokenizer, self.text_max_len, self.summary_max_len)

    def collate_fn(self, batch):
      texts = [item['text'] for item in batch]
      summaries = [item['summary'] for item in batch]
      ids = [item['id'] for item in batch]
      text_input_ids = pad_sequence([item['input_ids'].flatten() for item in batch], batch_first=True)
      text_attention_masks = pad_sequence([item['attention_mask'].flatten() for item in batch], batch_first=True)
      labels = pad_sequence([item['labels'] for item in batch], batch_first=True)
      labels_attention_masks = pad_sequence([item['labels_attention_mask'].flatten() for item in batch], batch_first=True)

      return {
          'text': texts,
          'summary': summaries,
          'id': ids,
          'input_ids': text_input_ids,
          'attention_mask': text_attention_masks,
          'labels': labels,
          'labels_attention_mask': labels_attention_masks
      }

    def train_dataloader(self):
      return DataLoader(self.train_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

    def val_dataloader(self):
      return DataLoader(self.val_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

    def test_dataloader(self):
      return DataLoader(self.test_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

## Model

In [ ]:
class SummaryModel(pl.LightningModule):

  def __init__(self, model):
    super().__init__()
    self.model = AutoModelForSeq2SeqLM.from_pretrained(model, return_dict=True)

  def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):

    output = self.model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        decoder_attention_mask=decoder_attention_mask,
                        labels=labels,
    )
    return output

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    decoder_attention_mask = batch["labels_attention_mask"]
    labels = batch["labels"]

    output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
    loss = output.loss
    self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    decoder_attention_mask = batch["labels_attention_mask"]
    labels = batch["labels"]

    output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
    loss = output.loss
    self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    decoder_attention_mask = batch["labels_attention_mask"]
    labels = batch["labels"]

    output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
    loss = output.loss
    self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    return AdamW(self.parameters(), lr=0.0001)

# Fine-Tuned models

### Pretrained T5 model

In [ ]:
# N_EPOCHS = 5
# BATCH_SIZE = 32

# tokenizer = AutoTokenizer.from_pretrained('t5-base')
# data_module = CustomDataModule(df_train, df_val, df_test, tokenizer, batch=BATCH_SIZE)
# model = SummaryModel('t5-base')

# checkpoint_callback = ModelCheckpoint(dirpath="checkpoints",
#                                       filename="best_checkpoints",
#                                       save_top_k=1,
#                                       verbose=True,
#                                       monitor="val_loss",
#                                       mode="min")

# trainer = pl.Trainer(callbacks=checkpoint_callback, max_epochs=N_EPOCHS, accelerator="gpu", enable_progress_bar=True)

In [ ]:
# trainer.fit(model, data_module)

# trained_model = SummaryModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
# trained_model.freeze()

In [8]:
tokenizer = AutoTokenizer.from_pretrained("sysresearch101/t5-large-finetuned-xsum-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("sysresearch101/t5-large-finetuned-xsum-cnn")

In [9]:
data_module = CustomDataModule(df_train, df_val, df_test, tokenizer, batch=8)
data_module.setup()

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

summaries, ids = [], []

with torch.no_grad():
    for batch in tqdm(data_module.test_dataloader()):
        summary_ids = model.generate(
          input_ids = batch['input_ids'].to(device),
          attention_mask=batch['attention_mask'].to(device),
          max_length=150,
          num_beams=2,
          repetition_penalty=2.5,
          length_penalty=1.0,
          early_stopping=True
      )

        summary_text = [tokenizer.decode(sum_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for sum_id in summary_ids]
        summaries.extend(summary_text)
        ids.extend(batch['id'])

  0%|          | 0/1437 [00:00<?, ?it/s]

In [16]:
data = {'id': ids, 'generated': summaries}
df_new = pd.DataFrame(data)
df_new.to_csv(f"results/t5_large_{dataset}.csv", index=False)

### BART

In [ ]:
# N_EPOCHS = 5
# BATCH_SIZE = 32

# tokenizer = AutoTokenizer.from_pretrained('bart-base')
# data_module = CustomDataModule(df_train, df_val, df_test, tokenizer, batch=BATCH_SIZE)
# model = SummaryModel('bart-base')

# checkpoint_callback = ModelCheckpoint(dirpath="checkpoints",
#                                       filename="best_checkpoints",
#                                       save_top_k=1,
#                                       verbose=True,
#                                       monitor="val_loss",
#                                       mode="min")

# trainer = pl.Trainer(callbacks=checkpoint_callback, max_epochs=N_EPOCHS, accelerator="gpu", enable_progress_bar=True)

In [ ]:
# trainer.fit(model, data_module)

# trained_model = SummaryModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
# trained_model.freeze()

In [21]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-xsum")

In [22]:
data_module = CustomDataModule(df_train, df_val, df_test, tokenizer, batch=8)
data_module.setup()

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

summaries, ids = [], []

dataloader = data_module.test_dataloader()

with torch.no_grad():
    for batch in tqdm(dataloader):
        summary_ids = model.generate(
          input_ids = batch['input_ids'].to(device),
          attention_mask=batch['attention_mask'].to(device),
          num_beams=6,
          length_penalty=2.0,
          no_repeat_ngram_size=4,
          min_length=10,
          max_length=60,
          early_stopping=True
        )

        summary_text = [tokenizer.decode(sum_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for sum_id in summary_ids]
        summaries.extend(summary_text)
        ids.extend(batch['id'])

  0%|          | 0/1437 [00:00<?, ?it/s]

In [25]:
data = {'id': ids, 'generated': summaries}
df_new = pd.DataFrame(data)
df_new.to_csv(f"results/bart_large_{dataset}.csv", index=False)

# OPEN-AI Models


In [3]:
openai.api_key = ''

### text-davinci-003

In [ ]:
def format_prompt(example_text, input_text):
    prompt = """System: You are an extractive summarizer that follows the output pattern.
            User: The following examples are successful extractive summarization instances: """ + example_text + """.
            Please summarize the following document. Document: + """ + str(input_text) + """,output: """
    return prompt

def generate_summary_davinci003(input_list, example_text):
  sum_dict = {"text":[],"summary":[]}

  for i, input_text in enumerate(tqdm(input_list)):
    # if i == 2:
    #   break

    sum_dict['text'].append(input_text)
    prompt = format_prompt(example_text, input_text)

    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens=128,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )
    sum_dict["summary"].append(response['choices'][0]['text'])

  sum_df = pd.DataFrame(sum_dict)
  sum_df.to_csv(f"results/davinci003_{dataset}.csv",index=False)

In [ ]:
text_inputs = df_test['text'].tolist()

random_rows = df_train.sample(n=2)
examples_prompt = ""
for _, row in random_rows.iterrows():
  examples_prompt += "input: " + row['text'] + ",output: " + row['summary'].replace('\n', '') + "\n"

# print(examples_prompt + " \n\n Total wordcount examples: " + str(len(examples_prompt)))

In [ ]:
# Ensure max token length of prompt is below 4097 token boundary (Note: Take into account max_tokens in 'openai.Completion.create' above)
encoding = tiktoken.encoding_for_model("text-davinci-003")
token_len = []
for text in text_inputs:
  input = format_prompt(examples_prompt, text)
  tokens = encoding.encode(input)
  token_len.append(len(tokens))

max(token_len)

4424

In [ ]:
generate_summary_davinci003(text_inputs, examples_prompt)

  0%|          | 0/11490 [00:00<?, ?it/s]

### get-3.5-turbo

In [14]:
text_inputs = df_test['text'].tolist()

tiktoken_enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
token_len = []
for text in tqdm(text_inputs):
  tokens = tiktoken_enc.encode(str(text))
  token_len.append(len(tokens))

print(f"max token length: {max(token_len)}, total tokens: {sum(token_len)}")

100%|██████████| 11334/11334 [00:05<00:00, 2035.46it/s]

max token length: 15255, total tokens: 5395732


In [15]:
tiktoken_enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

def summmaries_to_chunks(summary_text):
  chunks = [[]]
  chunk_token_count = 0

  # split sentences
  sentences = nlp(summary_text)

  for sentence in sentences.sents:
    # calculate token size using tiktoken library
    token_count = len(tiktoken_enc.encode(sentence.text))
    chunk_token_count += token_count

    # Check if token count is above the 4097 limit.
    # Else add new sublist and reset count for new chunk.
    if chunk_token_count > 3900:
      chunks.append([])
      chunk_token_count = token_count

    # Add text to last chunk
    chunks[len(chunks)-1].append(sentence.text)
  
  return chunks

In [21]:
def generate_summary_gpt35(input_list, chunking=False):
  sum_dict = {"text":[],"summary":[]}
  for i, input_text in enumerate(tqdm(input_list)):
    # if i == 2:
    #   break

    sum_dict['text'].append(input_text)

    if chunking:
      chunks = summmaries_to_chunks(input_text)
    else:
      chunks = [input_text]
    chunk_summaries = []

    for chunk_text in chunks:
      prompt_text = " ".join(chunk_text)
      response = openai.ChatCompletion.create(
                  model= "gpt-3.5-turbo",
                  messages= [{"role":"system", "content":"You are assistant who replies with a clear and concise summary for every text."},
                            {"role": "user", "content":"Summarize the following text in max 5 sentences: " + prompt_text}],
                  temperature=0.5,
                  max_tokens=128,
                  top_p=1.0,
                  frequency_penalty=0.0,
                  presence_penalty=0.0
              )
      chunk_summaries.append(response['choices'][0]['message']['content'])

    summary = " ".join(chunk_summaries)
    sum_dict["summary"].append(summary)
  return pd.DataFrame(sum_dict)

In [22]:
gpt35_df = generate_summary_gpt35(text_inputs, True)
gpt35_df.to_csv(f"results/davinci003_{dataset}.csv",index=False)
gpt35_df.head()

  0%|          | 0/11334 [00:00<?, ?it/s]

  0%|          | 3/11334 [00:13<13:57:51,  4.44s/it]


RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-BHcxurZhndAcsy31biIcg4gj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

# Copy results to Google Drive

In [26]:
%cp -r results/. ./drive/MyDrive/Grounding_LM/results/